In [1]:
import pandas as pd
import numpy as np
import re

**FASE 1. CARGA DATASETS**

In [2]:
# Los datasets que se usan en este notebook son:

# table_1.csv
# Custom-Regions-Tableau-Example.csv
# Flujo_africa_1.csv
# Flujo_latinoamerica_1.csv
# Flujo_europa_1.csv
# Flujo_asia_1.csv
# data_refugiados.csv

# Se encuentran en la carpeta 'datasets_notebook' dentro de la carpeta 'datasets_migrations'

In [3]:
# Especificamos a partir de que línea del archivo csv comienzan los headers de las variables. El resto de líneas son ignoradas
numero_linea = 5
# Especificamos el separador como ";" y aplicamos un encoding para evitar problemas asociados a la decodificación de símbolos.
df = pd.read_csv("table_1.csv", sep= ";", skiprows=range(0, numero_linea), header=numero_linea, encoding='latin-1')
# 
print(f"El dataset cuenta con {df.shape[0]} filas y {df.shape[1]} columnas")
# Mostramos por pantalla los 5 primeros resultados
df.tail(5)

El dataset cuenta con 37051 filas y 28 columnas


,Index,"Region, development group, country or area of destination",Notes of destination,Location code of destination,Type of data of destination,"Region, development group, country or area of origin",Location code of origin,1990,1995,2000,...,2010.1,2015.1,2020.1,1990.2,1995.2,2000.2,2005.2,2010.2,2015.2,2020.2
37046,37047,Wallis and Futuna Islands*,41.0,876,B R,French Polynesia*,258,18,23,28,...,10,8,5,8,11,13,16,9,7,5
37047,37048,Wallis and Futuna Islands*,41.0,876,B R,Mayotte*,175,35,44,52,...,23,26,18,16,20,24,28,18,12,9
37048,37049,Wallis and Futuna Islands*,41.0,876,B R,New Caledonia*,540,843,975,1 152,...,548,508,509,410,480,574,598,545,513,512
37049,37050,Wallis and Futuna Islands*,41.0,876,B R,Vanuatu,548,152,165,195,...,70,38,39,73,80,96,97,68,34,28
37050,37051,Wallis and Futuna Islands*,41.0,876,B R,Other,2003,27,36,43,...,13,12,13,12,16,20,25,12,15,13


**FASE 2. PREPROCESADO**

In [4]:
# Creamos una expresión regular para identificar las columnas que cumplen el formato deseado. 
# La expresión es cuatro digitos \d{4} seguidos de un punto \. y 1 ó 2 (1|2). 
expresion_regular = r'\d{4}\.(1|2)'

# Creamos una lista con el nombre de las columnas que cumplen con la expresión regular indicada
columns_delete = df.filter(regex=expresion_regular, axis=1).columns
# Lo convertimos a lista
columns_delete = columns_delete.tolist()
# A la lista de columnas a eliminar añadimos: 'Index', 'Notes of destination', 'Location code of destination', 'Type of data of destination', 'Location code of origin'
# Usamos el operador +=, aunque podíamos haber usado lista_actual.extend(lista_de_columnas_a_añadir)
columns_delete += ['Index', 'Notes of destination', 'Location code of destination', 'Type of data of destination', 'Location code of origin']
# Eliminamos del dataset las columnas contenidas en la lista creada
df_filter = df.drop(columns=columns_delete)

# Vamos a cambiar el nombre de las columnas 'Region, development group, country or area of destination' y 'Region, development group, country or area of origin' 
# por destino y origen respectivamente.

change_names = {
    'Region, development group, country or area of origin': 'origen',
    'Region, development group, country or area of destination': 'destino'
    }
    
# Aplicamos los cambios de nombre utilizando el método rename()
df_filter = df_filter.rename(columns=change_names)

# También cambiamos el orden de las columnas destino y origen
df_filter = df_filter[['origen', 'destino', '1990', '1995', '2000','2005','2010','2015', '2020']]

# Mostramos las columnas filtradas
df_filter.head(10)

,origen,destino,1990,1995,2000,2005,2010,2015,2020
0,WORLD,WORLD,152 986 157,161 289 976,173 230 585,191 446 828,220 983 187,247 958 644,280 598 105
1,Sub-Saharan Africa,WORLD,15 334 807,16 488 973,15 638 014,17 274 985,19 942 724,24 197 211,28 284 538
2,Northern Africa and Western Asia,WORLD,16 365 218,18 069 437,19 437 054,21 867 791,25 313 711,32 606 047,37 563 820
3,Central and Southern Asia,WORLD,31 104 020,29 069 983,30 897 694,33 347 690,41 082 388,46 481 130,51 229 549
4,Eastern and South-Eastern Asia,WORLD,14 774 419,17 860 923,21 440 854,25 556 983,30 977 206,35 205 590,38 400 740
5,Latin America and the Caribbean,WORLD,15 273 399,19 669 704,24 628 700,29 338 206,34 637 650,36 206 000,42 890 481
6,Oceania (excluding Australia and New Zealand),WORLD,272 517,313 061,375 176,407 138,441 903,495 220,565 281
7,Australia and New Zealand,WORLD,689 030,758 234,871 660,984 280,1 149 265,1 256 202,1 404 924
8,Europe and Northern America,WORLD,50 532 413,51 582 164,52 877 703,54 849 090,58 681 104,60 592 402,67 601 621
9,Developed regions,WORLD,51 826 246,52 993 691,54 458 296,56 551 836,60 587 161,62 613 012,69 815 370


In [5]:
# Comprobamos los tipos de datos de cada columna
print(f"Los tipos de cada variable son:\n{df_filter.dtypes}")

# Como vemos las columnas de los años no son adecuadas, los número están separados por espacios en blanco ('152 986 157') y no son por defecto de tipo -int- 
# Especificamos la lista de columnas que queremos convertir a tipo int
columns_to_int = ['1990', '1995', '2000', '2005', '2010', '2015', '2020']

# Creamos una copia del dataframe para que los pasos de este notebook se puedan repetir sin dar error
df_copy = df_filter.copy()

# Eliminamos los espacios en blanco entre los números
df_copy[columns_to_int] = df_copy[columns_to_int].apply(lambda x: x.str.replace(' ', ''))

# Reemplazamos cualquier signo que no sea un número por un cero, que consideraremos como NaN
df_copy[columns_to_int] = df_copy[columns_to_int].replace('[^\d]+', '0', regex=True)

# Los convertimos a tipo int
df_copy[columns_to_int] = df_copy[columns_to_int].astype(int)

# Mostrar el DataFrame resultante con las columnas convertidas a tipo entero
print(f"\nLos tipos finales de cada variable son:\n {df_copy.dtypes}")

# Mostramos los cinco primero resultados
df_copy.head(5)

Los tipos de cada variable son:
origen     object
destino    object
1990       object
1995       object
2000       object
2005       object
2010       object
2015       object
2020       object
dtype: object

Los tipos finales de cada variable son:
 origen     object
destino    object
1990        int64
1995        int64
2000        int64
2005        int64
2010        int64
2015        int64
2020        int64
dtype: object


,origen,destino,1990,1995,2000,2005,2010,2015,2020
0,WORLD,WORLD,152986157,161289976,173230585,191446828,220983187,247958644,280598105
1,Sub-Saharan Africa,WORLD,15334807,16488973,15638014,17274985,19942724,24197211,28284538
2,Northern Africa and Western Asia,WORLD,16365218,18069437,19437054,21867791,25313711,32606047,37563820
3,Central and Southern Asia,WORLD,31104020,29069983,30897694,33347690,41082388,46481130,51229549
4,Eastern and South-Eastern Asia,WORLD,14774419,17860923,21440854,25556983,30977206,35205590,38400740


In [6]:
# Eliminamos los espacios en blanco al principio y final de las columnas origen y destino
df_copy = df_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Eliminamos los asteriscos que algunos nombres tienen
df_copy = df_copy.replace(r'\*', '', regex=True)

In [7]:
# Creación de la columna Year y Num_migrantes
# Utilizamos el método melt() para convertir las columnas de años en una única columna llamada "year"
# Fuente: https://pandas.pydata.org/docs/reference/api/pandas.melt.html

df_melt = df_copy.melt(id_vars=['origen', 'destino'], value_vars=['1990', '1995', '2000', '2005', '2010', '2015', '2020'], var_name='year', value_name='num_migrantes')

# Mostramos los cinco primeros registros del DataFrame resultante
df_melt.head(5)

,origen,destino,year,num_migrantes
0,WORLD,WORLD,1990,152986157
1,Sub-Saharan Africa,WORLD,1990,15334807
2,Northern Africa and Western Asia,WORLD,1990,16365218
3,Central and Southern Asia,WORLD,1990,31104020
4,Eastern and South-Eastern Asia,WORLD,1990,14774419


**FASE 3. CREACION DATASETS**


**Creación dataset data_continentes**

In [8]:
# Creamos la lista con los nombres de los continentes que usaremos
continentes = ['OCEANIA', 'NORTHERN AMERICA', 'LATIN AMERICA AND THE CARIBBEAN', 'ASIA', 'AFRICA','EUROPE']

# Seleccionamos solo las filas donde origen o destino estén en la lista de continentes que hemos especificado. Para ello usamos la función isin()
# Fuente: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html

df_continentes = df_melt[df_melt['origen'].isin(continentes) & df_melt['destino'].isin(continentes)]
df_continentes.head(10)

,origen,destino,year,num_migrantes
5964,AFRICA,AFRICA,1990,13431488
5970,ASIA,AFRICA,1990,492107
5976,EUROPE,AFRICA,1990,622122
5981,LATIN AMERICA AND THE CARIBBEAN,AFRICA,1990,10188
5985,NORTHERN AMERICA,AFRICA,1990,58134
5986,OCEANIA,AFRICA,1990,9090
12025,AFRICA,ASIA,1990,1777604
12031,ASIA,ASIA,1990,35481900
12037,EUROPE,ASIA,1990,7632934
12042,LATIN AMERICA AND THE CARIBBEAN,ASIA,1990,118597


In [9]:
# Agregamos los valores de latitud y longitud como dos columnas mas

coordenadas = {
    'AFRICA': {'longitud': 17, 'latitud': 8},
    'ASIA': {'longitud': 95, 'latitud': 35},
    'EUROPE': {'longitud': 15, 'latitud': 52},
    'LATIN AMERICA AND THE CARIBBEAN': {'longitud': -85, 'latitud': 15},
    'NORTHERN AMERICA': {'longitud': -100, 'latitud': 45},
    'OCEANIA': {'longitud': 135, 'latitud': -20}
}

# Agregamos columnas de longitud y latitud al DataFrame para cada uno de los países de origen y destino
df_continentes_copy = df_continentes.copy()

df_continentes_copy.loc[:, 'lon_origen'] = df_continentes_copy['origen'].map(lambda x: coordenadas.get(x, {}).get('longitud'))
df_continentes_copy.loc[:, 'lat_origen'] = df_continentes_copy['origen'].map(lambda x: coordenadas.get(x, {}).get('latitud'))
df_continentes_copy.loc[:, 'lon_destino'] = df_continentes_copy['destino'].map(lambda x: coordenadas.get(x, {}).get('longitud'))
df_continentes_copy.loc[:, 'lat_destino'] = df_continentes_copy['destino'].map(lambda x: coordenadas.get(x, {}).get('latitud'))

# Reordenamos las columnas
column_order = ['origen', 'lon_origen', 'lat_origen', 'destino', 'lon_destino', 'lat_destino', 'year', 'num_migrantes']
df_continentes = df_continentes_copy.reindex(columns=column_order)

# Convertimos la columna 'year' a tipo int64
df_continentes['year'] = df_continentes['year'].astype('int64')

df_continentes.head(5)


,origen,lon_origen,lat_origen,destino,lon_destino,lat_destino,year,num_migrantes
5964,AFRICA,17,8,AFRICA,17,8,1990,13431488
5970,ASIA,95,35,AFRICA,17,8,1990,492107
5976,EUROPE,15,52,AFRICA,17,8,1990,622122
5981,LATIN AMERICA AND THE CARIBBEAN,-85,15,AFRICA,17,8,1990,10188
5985,NORTHERN AMERICA,-100,45,AFRICA,17,8,1990,58134


In [10]:
# Contamos valores cero en el DataFrame
valores_cero = (df_continentes == 0).sum()
# Mostramos los valores cero por columna
valores_cero

origen           0
lon_origen       0
lat_origen       0
destino          0
lon_destino      0
lat_destino      0
year             0
num_migrantes    0
dtype: int64

In [11]:
# A continuación vamos a adjuntar a nuestro dataframe el siguiente, que incluye por años y continente el porcentaje de la población que emigró

data = {
    'año': [1990, 1995, 2000, 2005, 2010, 2015, 2020],
    'AFRICA': [1.234, 1.078, 0.749, 0.675, 0.723, 0.831, 0.897],
    'ASIA': [1.765, 1.693, 1.746, 1.830, 2.093, 2.331, 2.475],
    'EUROPE': [6.621, 6.690, 6.853, 7.039, 7.429, 7.614, 8.463],
    'LATIN AMERICA AND THE CARIBBEAN': [3.448, 4.072, 4.719, 5.262, 5.857, 5.802, 6.558],
    'NORTHERN AMERICA': [1.001, 1.000, 1.008, 1.073, 1.156, 1.124, 1.173],
    'OCEANIA': [3.522, 3.645, 3.967, 4.130, 4.315, 4.394, 4.616]
}

df_porcentajes_emigrantes = pd.DataFrame(data)

# Utilizamos melt para convertir las columnas de continente en una sola columna
df_emigrantes_by_continent = df_porcentajes_emigrantes.melt(id_vars='año', var_name='continente', value_name='porcentaje_emigrantes_origen')



In [12]:
# Unimos los DataFrames utilizando las coincidencias de 'year' y 'origen'
df_joined = pd.merge(df_continentes, df_emigrantes_by_continent, left_on=['year', 'origen'], right_on=['año', 'continente'])

df_joined.head(5)

,origen,lon_origen,lat_origen,destino,lon_destino,lat_destino,year,num_migrantes,año,continente,porcentaje_emigrantes_origen
0,AFRICA,17,8,AFRICA,17,8,1990,13431488,1990,AFRICA,1.234
1,AFRICA,17,8,ASIA,95,35,1990,1777604,1990,AFRICA,1.234
2,AFRICA,17,8,EUROPE,15,52,1990,4682594,1990,AFRICA,1.234
3,AFRICA,17,8,LATIN AMERICA AND THE CARIBBEAN,-85,15,1990,19192,1990,AFRICA,1.234
4,AFRICA,17,8,NORTHERN AMERICA,-100,45,1990,548628,1990,AFRICA,1.234


In [13]:
# Eliminamos las columnas con información duplicada
df_continentes_final = df_joined.drop(['año', 'continente'], axis=1)

In [14]:
# Añadimos la población global en función del año
# Diccionario de población global por año
poblacion_global_dict = {
    1990: 5327231,
    1995: 5744213,
    2000: 6143494,
    2005: 6541907,
    2010: 6956824,
    2015: 7379797,
    2020: 7794799
}

# 'Mapeamos' los valores de población global al DataFrame
df_continentes_final['poblacion_global'] = df_continentes_final['year'].map(poblacion_global_dict)

# Añadimos los datos relativos a la población total de cada continente por año
data = {
    'año': [1990, 1995, 2000, 2005, 2010, 2015, 2020],
    'AFRICA': [630350, 717270, 810984, 916154, 1039304, 1182439, 1340598],
    'ASIA': [3226099, 3493087, 3741263, 3977986, 4209594, 4433475, 4641055],
    'EUROPE': [720858, 726994, 725558, 729288, 736413, 743059, 747636],
    'LATIN AMERICA AND THE CARIBBEAN': [442840, 483018, 521836, 557501, 591352, 623934, 653962],
    'NORTHERN AMERICA': [279785, 294454, 312427, 327287, 343287, 357031, 368870],
    'OCEANIA': [27299, 29390, 31425, 33690, 36873, 39859, 42678]
}

# Creamos una función para obtener el valor de la lista 'data' basado en 'origen' y 'year'
def obtener_valor(row):
    origen = row['origen']
    year = row['year']
    if origen in data.keys() and year in data['año']:
        idx = data['año'].index(year)
        return data[origen][idx]
    return None

# Creamos la columna 'valor_origen' en df2 utilizando la función obtener_valor
df_continentes_final['poblacion_total_origen'] = df_continentes_final.apply(obtener_valor, axis=1)


# Visualizamos los primeros cinco resultados
df_continentes_final.head(5)

,origen,lon_origen,lat_origen,destino,lon_destino,lat_destino,year,num_migrantes,porcentaje_emigrantes_origen,poblacion_global,poblacion_total_origen
0,AFRICA,17,8,AFRICA,17,8,1990,13431488,1.234,5327231,630350
1,AFRICA,17,8,ASIA,95,35,1990,1777604,1.234,5327231,630350
2,AFRICA,17,8,EUROPE,15,52,1990,4682594,1.234,5327231,630350
3,AFRICA,17,8,LATIN AMERICA AND THE CARIBBEAN,-85,15,1990,19192,1.234,5327231,630350
4,AFRICA,17,8,NORTHERN AMERICA,-100,45,1990,548628,1.234,5327231,630350


In [15]:
# Dimensiones del dataframe
df_continentes_final.shape

(252, 11)

**Creación dataset data_zona_geografica**

In [16]:
# Nos quedamos solo con zonas geográficas
zonas_geograficas = ['Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa', 'Central Asia', 'Eastern Asia', 'South-Eastern Asia',
                     'Southern Asia', 'Western Asia', 'Eastern Europe', 'Northern Europe', 'Southern Europe', 'Western Europe', 'Caribbean', 'Central America', 
                     'South America', 'NORTHERN AMERICA', 'OCEANIA']

# Comprobamos que todos los elementos de la lista creada tiene al menos una aparición en la columna origen. Es decir, que no ha habido problemas de escritura.
# Convertimos la lista en una Serie de Pandas
series_zonas_geograficas = pd.Series(zonas_geograficas)

# Encontramos los valores de la serie que no están en la columna 'origen'
valores_no_presentes = series_zonas_geograficas[~series_zonas_geograficas.isin(df_melt['origen'])]

# Mostramos los valores no presentes
valores_no_presentes

# Al estar vacia implica que hemos escrito adecuadamente los valores

Series([], dtype: object)

In [17]:
# Procedemos como en el anterior apartado
df_zonas_geograficas = df_melt[df_melt['origen'].isin(zonas_geograficas) & df_melt['destino'].isin(zonas_geograficas)]
df_zonas_geograficas.head(10)

,origen,destino,year,num_migrantes
6248,Eastern Africa,Eastern Africa,1990,4900387
6249,Middle Africa,Eastern Africa,1990,370968
6250,Northern Africa,Eastern Africa,1990,179281
6251,Southern Africa,Eastern Africa,1990,36745
6252,Western Africa,Eastern Africa,1990,5911
6254,Central Asia,Eastern Africa,1990,0
6255,Eastern Asia,Eastern Africa,1990,2568
6256,South-Eastern Asia,Eastern Africa,1990,104
6257,Southern Asia,Eastern Africa,1990,17476
6258,Western Asia,Eastern Africa,1990,357


In [18]:
# Añadimos coordenadas de latitud y longitud

coordenadas_zonas_geograficas = {
    'Eastern Africa': {'longitud': 39.0, 'latitud': -6.5},
    'Middle Africa': {'longitud': 17.0, 'latitud': 1.0},
    'Northern Africa': {'longitud': 18.0, 'latitud': 28.0},
    'Southern Africa': {'longitud': 24.0, 'latitud': -29.0},
    'Western Africa': {'longitud': -2.0, 'latitud': 16.0},
    'Central Asia': {'longitud': 63.0, 'latitud': 48.0},
    'Eastern Asia': {'longitud': 116.0, 'latitud': 39.0},
    'South-Eastern Asia': {'longitud': 108.0, 'latitud': 4.0},
    'Southern Asia': {'longitud': 78.0, 'latitud': 23.0},
    'Western Asia': {'longitud': 40.0, 'latitud': 33.0},
    'Eastern Europe': {'longitud': 30.0, 'latitud': 50.0},
    'Northern Europe': {'longitud': 15.0, 'latitud': 60.0},
    'Southern Europe': {'longitud': 15.0, 'latitud': 41.0},
    'Western Europe': {'longitud': 2.0, 'latitud': 48.0},
    'Caribbean': {'longitud': -75.0, 'latitud': 18.0},
    'Central America': {'longitud': -86.0, 'latitud': 15.0},
    'South America': {'longitud': -60.0, 'latitud': -10.0},
    'NORTHERN AMERICA': {'longitud': -100.0, 'latitud': 45.0},
    'OCEANIA': {'longitud': 140.0, 'latitud': -25.0}
}

# Agregamos columnas de longitud y latitud al DataFrame para cada uno de los países de origen y destino
df_zonas_geograficas = df_zonas_geograficas.copy()

df_zonas_geograficas.loc[:, 'lon_origen'] = df_zonas_geograficas['origen'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('longitud'))
df_zonas_geograficas.loc[:, 'lat_origen'] = df_zonas_geograficas['origen'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('latitud'))
df_zonas_geograficas.loc[:, 'lon_destino'] = df_zonas_geograficas['destino'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('longitud'))
df_zonas_geograficas.loc[:, 'lat_destino'] = df_zonas_geograficas['destino'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('latitud'))

# Reordenamos las columnas
column_order = ['origen', 'lon_origen', 'lat_origen', 'destino', 'lon_destino', 'lat_destino', 'year', 'num_migrantes']
df_zonas_geograficas = df_zonas_geograficas.reindex(columns=column_order)

df_zonas_geograficas.head(5)

,origen,lon_origen,lat_origen,destino,lon_destino,lat_destino,year,num_migrantes
6248,Eastern Africa,39.0,-6.5,Eastern Africa,39.0,-6.5,1990,4900387
6249,Middle Africa,17.0,1.0,Eastern Africa,39.0,-6.5,1990,370968
6250,Northern Africa,18.0,28.0,Eastern Africa,39.0,-6.5,1990,179281
6251,Southern Africa,24.0,-29.0,Eastern Africa,39.0,-6.5,1990,36745
6252,Western Africa,-2.0,16.0,Eastern Africa,39.0,-6.5,1990,5911


In [19]:
# Mostramos las dimensiones del dataframe creado
df_zonas_geograficas.shape

(2527, 8)

**Creación dataset data_paises**

In [20]:
# En una lista guardamos los primeros 22 valores de la variable origen, que se corresponde con valores que no son países
lista_to_ignore = df_copy['origen'].head(21).tolist()
lista_to_ignore

['WORLD',
 'Sub-Saharan Africa',
 'Northern Africa and Western Asia',
 'Central and Southern Asia',
 'Eastern and South-Eastern Asia',
 'Latin America and the Caribbean',
 'Oceania (excluding Australia and New Zealand)',
 'Australia and New Zealand',
 'Europe and Northern America',
 'Developed regions',
 'Less developed regions',
 'Less developed regions, excluding least developed countries',
 'Less developed regions, excluding China',
 'Least developed countries',
 'Land-locked Developing Countries (LLDC)',
 'Small island developing States (SIDS)',
 'High-income countries',
 'Middle-income countries',
 'Upper-middle-income countries',
 'Lower-middle-income countries',
 'Low-income countries']

In [21]:
# Definimos la lista de valores a excluir juntando las listas que hemos ido creando en los anteriores casos
valores_excluir = lista_to_ignore + zonas_geograficas + continentes

# Filtramos el DataFrame manteniendo solo los valores que no están en la lista
df_countries= df_melt[~(df_melt['origen'].isin(valores_excluir) | df_melt['destino'].isin(valores_excluir))]

# Mostramos el DataFrame filtrado
df_countries.head(10)

,origen,destino,year,num_migrantes
6554,Melanesia,Burundi,1990,0
6555,Micronesia,Burundi,1990,0
6556,Polynesia,Burundi,1990,0
6557,Belgium,Burundi,1990,642
6558,Democratic Republic of the Congo,Burundi,1990,36654
6559,France,Burundi,1990,321
6560,Kenya,Burundi,1990,186
6561,Rwanda,Burundi,1990,221943
6562,Uganda,Burundi,1990,1833
6563,United Republic of Tanzania,Burundi,1990,11912


In [22]:
# Eliminamos el país con valor -other-
df_countries = df_countries.loc[~(df_countries['origen'] == 'other') & ~(df_countries['destino'] == 'other')]

In [23]:
# Mostramos la lista de países creados. Como vemos necesitamos filtrar los países, seleccionamos los 192 reconocidos por la ONU
len(df_countries.origen.unique())

239

In [24]:
# Agregamos sus coordenadas geográficas
countries_coordenadas = {
    'Belgium': (50.503887, 4.469936),
    'Democratic Republic of the Congo': (-4.038333, 21.758664),
    'France': (46.603354, 1.888334),
    'Kenya': (-0.023559, 37.906193),
    'Rwanda': (-1.940278, 29.873888),
    'Uganda': (1.373333, 32.290275),
    'United Republic of Tanzania': (-6.369028, 34.888822),
    'Madagascar': (-18.87919, 47.507905),
    'Réunion': (-21.115141, 55.536384),
    'Ethiopia': (9.145, 40.489673),
    'Somalia': (5.152149, 46.199616),
    'Yemen': (15.552727, 48.516388),
    'Angola': (-11.202692, 17.873887),
    'Burundi': (-3.373056, 29.918886),
    'China': (35.86166, 104.195397),
    'Comoros': (-11.6455, 43.3333),
    'India': (20.593684, 78.96288),
    'Malawi': (-13.254308, 34.301525),
    'Mozambique': (-18.665695, 35.529562),
    'South Africa': (-30.559482, 22.937506),
    'Zimbabwe': (-19.015438, 29.154857),
    'South Sudan': (6.876991, 31.306978),
    'Sudan': (12.862807, 30.217636),
    'United Kingdom': (55.378051, -3.435973),
    'Zambia': (-13.133897, 27.849332),
    'Eritrea': (15.179384, 39.782334),
    'Djibouti': (11.825138, 42.590275),
    'Namibia': (-22.95764, 18.49041),
    'Congo': (-0.228021, 15.827659),
    'Bangladesh': (23.684994, 90.356331),
    'Canada': (56.130366, -106.346771),
    'Sri Lanka': (7.873054, 80.771797),
    'Germany': (51.165691, 10.451526),
    'Ireland': (53.41291, -8.24389),
    'Italy': (41.87194, 12.56738),
    'Japan': (36.204824, 138.252924),
    'Malaysia': (4.210484, 101.975766),
    'Pakistan': (30.375321, 69.345116),
    'Russian Federation': (61.52401, 105.318756),
    'Seychelles': (-4.679574, 55.491977),
    'Spain': (40.463667, -3.74922),
    'Switzerland': (46.818188, 8.227512),
    'United States of America': (37.09024, -95.712891),
    'Cabo Verde': (16.5388, -23.0418),
    'Lesotho': (-29.609988, 28.233608),
    'Nigeria': (9.081999, 8.675277),
    'Portugal': (39.399872, -8.224454),
    'Mauritius': (-20.348404, 57.552152),
    'Mayotte': (-12.8275, 45.166244),
    'Australia': (-25.274398, 133.775136),
    'Cuba': (21.521757, -77.781167),
    'Indonesia': (-0.789275, 113.921327),
    'Maldives': (3.202778, 73.22068),
    'Nepal': (28.394857, 84.124008),
    'Philippines': (12.879721, 121.774017),
    'Thailand': (15.870032, 100.992541),
    'Central African Republic': (6.611111, 20.939444),
    'Chad': (15.454166, 18.732207),
    'Egypt': (26.820553, 30.802498),
    'Libya': (26.3351, 17.228331),
    'Mali': (17.570692, -3.996166),
    'Algeria': (28.033886, 1.659626),
    'Denmark': (56.26392, 9.501785),
    'Gambia': (13.443182, -15.310139),
    'Ghana': (7.946527, -1.023194),
    'Greece': (39.074208, 21.824312),
    'Hungary': (47.162494, 19.503304),
    "Côte d'Ivoire": (7.539989, -5.54708),
    "Dem. People's Republic of Korea": (40.339852, 127.510093),
    'Republic of Korea': (35.907757, 127.766922),
    'Mauritania': (21.00789, -10.940835),
    'Netherlands': (52.132633, 5.291266),
    'Norway': (60.472024, 8.468946),
    'Senegal': (14.497401, -14.452362),
    'Sweden': (60.128161, 18.643501),
    'Togo': (8.619543, 0.824782),
    'Botswana': (-22.328474, 24.684866),
    'Eswatini': (-26.522503, 31.465866),
    'Cameroon': (7.369722, 12.354722),
    'Guinea': (9.945587, -9.696645),
    'Sao Tome and Principe': (0.18636, 6.613081),
    'Benin': (9.30769, 2.315834),
    'Equatorial Guinea': (1.650801, 10.267895),
    'Gabon': (-0.803689, 11.609444),
    'Liberia': (6.428055, -9.429499),
    'Niger': (17.607789, 8.081666),
    'Austria': (47.516231, 14.550072),
    'Cyprus': (35.126413, 33.429859),
    'Slovenia': (46.151241, 14.995463),
    'Turkey': (38.963745, 35.243322),
    'Lebanon': (33.854721, 35.862285),
    'Israel': (31.046051, 34.851612),
    'Morocco': (31.791702, -7.09262),
    'Syrian Arab Republic': (34.802075, 38.996815),
    'Tunisia': (33.886917, 9.537499),
    'Burkina Faso': (12.238333, -1.561593),
    'State of Palestine': (31.952162, 35.233154),
    'Iraq': (33.223191, 43.679291),
    'Jordan': (30.585164, 36.238414),
    'Kuwait': (29.31166, 47.481766),
    'Saudi Arabia': (23.885942, 45.079162),
    'Western Sahara': (24.215527, -12.885834),
    'Afghanistan': (33.93911, 67.709953),
    'Albania': (41.153332, 20.168331),
    'Antigua and Barbuda': (17.060816, -61.796428),
    'Azerbaijan': (40.143105, 47.576927),
    'Argentina': (-38.416097, -63.616672),
    'Bahamas': (25.03428, -77.39628),
    'Bahrain': (25.930414, 50.637772),
    'Armenia': (40.069099, 45.038189),
    'Bosnia and Herzegovina': (43.915886, 17.679076),
    'Brazil': (-14.235004, -51.92528),
    'Brunei Darussalam': (4.535277, 114.727669),
    'Bulgaria': (42.733883, 25.48583),
    'Myanmar': (21.916221, 95.956223),
    'Belarus': (53.709807, 27.953389),
    'Chile': (-35.675147, -71.542969),
    'Colombia': (4.570868, -74.297333),
    'Colombia': (4.570868, -74.297333),
    'Croatia': (45.1, 15.2),
    'Czechia': (49.817492, 15.472962),
    'Ecuador': (-1.831239, -78.183406),
    'El Salvador': (13.794185, -88.89653),
    'Estonia': (58.595272, 25.013607),
    'Finland': (61.92411, 25.748151),
    'Gibraltar': (36.137741, -5.345374),
    'Guatemala': (15.783471, -90.230759),
    'Iran (Islamic Republic of)': (32.427908, 53.688046),
    'Kazakhstan': (48.019573, 66.923684),
    'Kyrgyzstan': (41.20438, 74.766098),
    'Mexico': (23.634501, -102.552784),
    'Monaco': (43.750298, 7.412841),
    'Mongolia': (46.862496, 103.846656),
    'Republic of Moldova': (47.411631, 28.369885),
    'Oman': (21.512583, 55.923255),
    'New Zealand': (-40.900557, 174.885971),
    'Poland': (51.919438, 19.145136),
    'Guinea-Bissau': (11.803749, -15.180413),
    'Qatar': (25.354826, 51.183884),
    'Romania': (45.943161, 24.96676),
    'Sierra Leone': (8.460555, -11.779889),
    'Singapore': (1.352083, 103.819836),
    'Slovakia': (48.669026, 19.699024),
    'Tajikistan': (38.861034, 71.276093),
    'United Arab Emirates': (23.424076, 53.847818),
    'Turkmenistan': (38.969719, 59.556278),
    'Ukraine': (48.379433, 31.16558),
    'North Macedonia': (41.608635, 21.745275),
    'Uzbekistan': (41.377491, 64.585262),
    'Venezuela (Bolivarian Republic of)': (6.42375, -66.58973),
    'Luxembourg': (49.815273, 6.129583),
    'Barbados': (13.193887, -59.543198),
    'Bermuda': (32.321384, -64.75737),
    'Bolivia (Plurinational State of)': (-16.290154, -63.588653),
    'British Virgin Islands': (18.420695, -64.639968),
    'Fiji': (-16.578193, 179.414413),
    'Guyana': (4.860416, -58.93018),
    'Iceland': (64.963051, -19.020835),
    'Jamaica': (18.109581, -77.297508),
    'Latvia': (56.879635, 24.603189),
    'Lithuania': (55.169438, 23.881275),
    'Malta': (35.937496, 14.375416),
    'Nauru': (-0.522778, 166.931503),
    'Austria': (47.516231, 14.550072),
    'Cyprus': (35.126413, 33.429859),
    'Slovenia': (46.151241, 14.995463),
    'Turkey': (38.963745, 35.243322),
    'Lebanon': (33.854721, 35.862285),
    'Northern Mariana Islands': (15.0979, 145.6739),
    'Palau': (7.51498, 134.5825),
    'Cambodia': (12.5657, 104.991),
    'Panama': (8.538, -80.7821),
    'Dominica': (15.414999, -61.370976),
    'Uruguay': (-32.5228, -55.7658),
    'Serbia': (44.0165, 21.0059),
    'Peru': (-9.190, -75.0152),
    'Solomon Islands': (-9.6457, 160.1562),
    'Liechtenstein': (47.166, 9.5554),
    'Viet Nam': (14.0583, 108.2772),
    'Honduras': (15.2, -86.2419),
    'Bhutan': (27.5142, 90.4336),
    'Haiti': (18.9712, -72.2852),
    'Nicaragua': (12.8654, -85.2072),
    'United States Virgin Islands': (18.3358, -64.8963),
    'Trinidad and Tobago': (10.6918, -61.2225),
    'Belize': (17.1899, -88.4976),
    'Costa Rica': (9.7489, -83.7534),
    'Andorra': (42.5463, 1.6016),
    'Paraguay': (-23.442503, -58.443832),
    'Papua New Guinea': (-6.314993, 143.95555),
    'Niue': (-19.0544, -169.8672),
    'Saint Kitts and Nevis': (17.3578, -62.7822),
    'Saint Helena': (-15.965, -5.7089),
    'Marshall Islands': (7.1315, 171.1845),
    'Saint Vincent and the': (12.9843, -61.2872),
    'Timor-Leste': (-8.8742, 125.7275),
    'Kiribati': (1.8709, -157.362),
    'Dominican Republic': (18.7357, -70.1627)
}
print(f"Lista países con coordenadas {len(countries_coordenadas)}")

Lista países con coordenadas 192


In [25]:
# Creamos la lista con los países elegidos
choosen_countries = [
    "Belgium",
    "Democratic Republic of the Congo",
    "France",
    "Kenya",
    "Rwanda",
    "Uganda",
    "United Republic of Tanzania",
    "Madagascar",
    "Réunion",
    "Ethiopia",
    "Somalia",
    "Yemen",
    "Angola",
    "Burundi",
    "China",
    "Comoros",
    "India",
    "Malawi",
    "Mozambique",
    "South Africa",
    "Zimbabwe",
    "South Sudan",
    "Sudan",
    "United Kingdom",
    "Zambia",
    "Eritrea",
    "Djibouti",
    "Namibia",
    "Congo",
    "Bangladesh",
    "Canada",
    "Sri Lanka",
    "Germany",
    "Ireland",
    "Italy",
    "Japan",
    "Malaysia",
    "Pakistan",
    "Russian Federation",
    "Seychelles",
    "Spain",
    "Switzerland",
    "United States of America",
    "Cabo Verde",
    "Lesotho",
    "Nigeria",
    "Portugal",
    "Mauritius",
    "Mayotte",
    "Australia",
    "Cuba",
    "Indonesia",
    "Maldives",
    "Nepal",
    "Philippines",
    "Thailand",
    "Central African Republic",
    "Chad",
    "Egypt",
    "Libya",
    "Mali",
    "Algeria",
    "Denmark",
    "Gambia",
    "Ghana",
    "Greece",
    "Hungary",
    "Côte d'Ivoire",
    "Dem. People's Republic of Korea",
    "Republic of Korea",
    "Mauritania",
    "Netherlands",
    "Norway",
    "Senegal",
    "Sweden",
    "Togo",
    "Botswana",
    "Eswatini",
    "Cameroon",
    "Guinea",
    "Sao Tome and Principe",
    "Benin",
    "Equatorial Guinea",
    "Gabon",
    "Liberia",
    "Niger",
    "Austria",
    "Cyprus",
    "Slovenia",
    "Turkey",
    "Lebanon",
    "Israel",
    "Morocco",
    "Syrian Arab Republic",
    "Tunisia",
    "Burkina Faso",
    "State of Palestine",
    "Iraq",
    "Jordan",
    "Kuwait",
    "Saudi Arabia",
    "Western Sahara",
    "Afghanistan",
    "Albania",
    "Antigua and Barbuda",
    "Azerbaijan",
    "Argentina",
    "Bahamas",
    "Bahrain",
    "Armenia",
    "Bosnia and Herzegovina",
    "Brazil",
    "Brunei Darussalam",
    "Bulgaria",
    "Myanmar",
    "Belarus",
    "Chile",
    "Colombia",
    "Croatia",
    "Czechia",
    "Ecuador",
    "El Salvador",
    "Estonia",
    "Finland",
    "Gibraltar",
    "Guatemala",
    "Iran (Islamic Republic of)",
    "Kazakhstan",
    "Kyrgyzstan",
    "Mexico",
    "Monaco",
    "Mongolia",
    "Republic of Moldova",
    "Oman",
    "New Zealand",
    "Poland",
    "Guinea-Bissau",
    "Qatar",
    "Romania",
    "Sierra Leone",
    "Singapore",
    "Slovakia",
    "Tajikistan",
    "United Arab Emirates",
    "Turkmenistan",
    "Ukraine",
    "North Macedonia",
    "Uzbekistan",
    "Venezuela (Bolivarian Republic of)",
    "Luxembourg",
    "Barbados",
    "Bermuda",
    "Bolivia (Plurinational State of)",
    "British Virgin Islands",
    "Fiji",
    "Guyana",
    "Iceland",
    "Jamaica",
    "Latvia",
    "Lithuania",
    "Malta",
    "Nauru",
    "Nicaragua",
    "Niue",
    "Northern Mariana Islands",
    "Panama",
    "Papua New Guinea",
    "Paraguay",
    "Peru",
    "Saint Helena",
    "Saint Kitts and Nevis",
    "Serbia",
    "Trinidad and Tobago",
    "Uruguay",
    "Viet Nam",
    "Timor-Leste",
    "United States Virgin Islands",
    "Andorra",
    "Bhutan",
    "Belize",
    "Solomon Islands",
    "Cambodia",
    "Costa Rica",
    "Dominica",
    "Dominican Republic",
    "Kiribati",
    "Haiti",
    "Honduras",
    "Liechtenstein",
    "Marshall Islands",
    "Palau",
    "Saint Vincent and the"
] 

# Filtramos los registros cuyos valores de origen y destino estén en la lista de países seleccionados
df_countries = df_countries[df_countries['origen'].isin(choosen_countries) & df_countries['destino'].isin(choosen_countries)]

In [26]:
# Agregamos columnas de longitud y latitud al DataFrame para cada uno de los países de origen y destino
df_countries  = df_countries .copy()

df_countries.loc[:, 'lon_origen'] = df_countries['origen'].map(lambda x: countries_coordenadas.get(x, (None, None))[1])
df_countries.loc[:, 'lat_origen'] = df_countries['origen'].map(lambda x: countries_coordenadas.get(x, (None, None))[0])
df_countries.loc[:, 'lon_destino'] = df_countries['destino'].map(lambda x: countries_coordenadas.get(x, (None, None))[1])
df_countries.loc[:, 'lat_destino'] = df_countries['destino'].map(lambda x: countries_coordenadas.get(x, (None, None))[0])

# Reordenamos las columnas
column_order = ['origen', 'lon_origen', 'lat_origen', 'destino', 'lon_destino', 'lat_destino', 'year', 'num_migrantes']
df_countries = df_countries.reindex(columns=column_order)

df_countries.head(5)

,origen,lon_origen,lat_origen,destino,lon_destino,lat_destino,year,num_migrantes
6557,Belgium,4.469936,50.503887,Burundi,29.918886,-3.373056,1990,642
6558,Democratic Republic of the Congo,21.758664,-4.038333,Burundi,29.918886,-3.373056,1990,36654
6559,France,1.888334,46.603354,Burundi,29.918886,-3.373056,1990,321
6560,Kenya,37.906193,-0.023559,Burundi,29.918886,-3.373056,1990,186
6561,Rwanda,29.873888,-1.940278,Burundi,29.918886,-3.373056,1990,221943


In [27]:
# Mostramos la dimensiones del dataframe resultante
df_countries.shape

(73311, 8)

**Creación dataset data_economia**

In [28]:
# Creamos una lista con los valores asociados a lo económico
development = ['Developed regions',
 'Less developed regions',
 'Less developed regions, excluding least developed countries',
 'Less developed regions, excluding China',
 'Least developed countries',
 'Land-locked Developing Countries (LLDC)',
 'Small island developing States (SIDS)',
 'High-income countries',
 'Middle-income countries',
 'Upper-middle-income countries',
 'Lower-middle-income countries',
 'Low-income countries'] 

# Filtramos el DataFrame manteniendo solo los valores que no están en la lista zonas_geográficas para la variable origen y development para la variable destino
df_economia = df_melt[(df_melt['origen'].isin(zonas_geograficas) & df_melt['destino'].isin(development))]
df_economia.head(10)

,origen,destino,year,num_migrantes
2569,Eastern Africa,Developed regions,1990,863906
2570,Middle Africa,Developed regions,1990,410433
2571,Northern Africa,Developed regions,1990,3161800
2572,Southern Africa,Developed regions,1990,229362
2573,Western Africa,Developed regions,1990,726622
2575,Central Asia,Developed regions,1990,5115880
2576,Eastern Asia,Developed regions,1990,3844369
2577,South-Eastern Asia,Developed regions,1990,3779787
2578,Southern Asia,Developed regions,1990,2747367
2579,Western Asia,Developed regions,1990,6077240


In [29]:
# Diccionario de mapeo de regiones a continentes
mapeo_continentes = {
    'Eastern Africa': 'África',
    'Middle Africa': 'África',
    'Northern Africa': 'África',
    'Southern Africa': 'África',
    'Western Africa': 'África',
    'Central Asia': 'Asia',
    'Eastern Asia': 'Asia',
    'South-Eastern Asia': 'Asia',
    'Southern Asia': 'Asia',
    'Western Asia': 'Asia',
    'Eastern Europe': 'Europa',
    'Northern Europe': 'Europa',
    'Southern Europe': 'Europa',
    'Western Europe': 'Europa',
    'Caribbean': 'Latinoamerica',
    'Central America': 'Latinoamerica',
    'South America': 'Latinoamerica',
    'NORTHERN AMERICA': 'América',
    'OCEANIA': 'Oceanía'
}

# Aplicar el mapeo de continentes
df_economia['continentes'] = df_economia['origen'].map(mapeo_continentes)

data_economia_development_final = df_economia[df_economia['destino'].isin(['Developed regions', 'Less developed regions'])]

data_economia_income_final = df_economia[df_economia['destino'].isin(['High-income countries', 'Middle-income countries',
       'Low-income countries'])]

<ipython-input-29-c3d4119520ff>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_economia['continentes'] = df_economia['origen'].map(mapeo_continentes)


**Creación dataset data_economia_continentes**

In [30]:
# Creamos una lista con los valores asociados a lo económico
development = ['Developed regions',
 'Less developed regions',
 'Less developed regions, excluding least developed countries',
 'Less developed regions, excluding China',
 'Least developed countries',
 'Land-locked Developing Countries (LLDC)',
 'Small island developing States (SIDS)',
 'High-income countries',
 'Middle-income countries',
 'Upper-middle-income countries',
 'Lower-middle-income countries',
 'Low-income countries'] 

# Filtramos el DataFrame manteniendo solo los valores que no están en la lista zonas_geográficas para la variable origen y development para la variable destino
df_economia_continentes_origen = df_melt[(df_melt['origen'].isin(continentes) & df_melt['destino'].isin(development))]

# Creamos las listas con los valores que usaremos para las categorías de income y development
income_countries = ['High-income countries', 'Middle-income countries', 'Low-income countries']
development_regions = ['Developed regions', 'Less developed regions', 'Least developed countries']

# Filtramos los valores de la columna income en función de los valores de las listas que hemos creado. Si un elemento de la columna 'destino' pertenece a 
# la lista, entonces guardamos su valor en la columna income o development, en caso contrario asignamos nan. 
df_economia_continentes_origen['income'] = np.where(df_economia_continentes_origen['destino'].isin(income_countries),
                                                   df_economia_continentes_origen['destino'],
                                                   np.nan)

df_economia_continentes_origen['development'] = np.where(df_economia_continentes_origen['destino'].isin(development_regions),
                                                   df_economia_continentes_origen['destino'],
                                                  np.nan)
# Eliminamos la columna -destino-
df_economia_continentes_origen = df_economia_continentes_origen.drop('destino', axis=1)

#df_economia_continentes_origen.loc[:, 'lon_origen'] = df_economia_continentes_origen['origen'].map(lambda x: coordenadas.get(x, {}).get('longitud'))
#df_economia_continentes_origen.loc[:, 'lat_origen'] = df_economia_continentes_origen['origen'].map(lambda x: coordenadas.get(x, {}).get('latitud'))

# Convertimos los valores a minúsculas y capitalizamos la primera letra. Estos pasos los hacemos para después en tableu poder usar una base de datos externa
# que nos permita colorear los continentes. Adaptamos nuestros valores a los de esa base para continentes. 
df_economia_continentes_origen['origen'] = df_economia_continentes_origen['origen'].str.lower().str.capitalize()
df_economia_continentes_origen['origen'] = df_economia_continentes_origen['origen'].replace('Latin america and the caribbean', 'South America')
df_economia_continentes_origen['origen'] = df_economia_continentes_origen['origen'].replace('Northern america', 'North America')

# Creamos un dataframe solo con los valores de income (cuando income es distinto de nan)
df_economia_continentes_origen = df_economia_continentes_origen.loc[df_economia_continentes_origen['income'] != 'nan']
# Borramos la columna development
df_economia_continentes_income = df_economia_continentes_origen.drop('development', axis=1).dropna()
# Creamos un dataframe solo con los valores de development (cuando development es distinto de nan)
df_economia_continentes_origen.loc[df_economia_continentes_origen['income'] != 'nan']
# Borramos la columna income
df_economia_continentes_development = df_economia_continentes_origen = df_economia_continentes_origen.drop('income', axis=1).dropna()


<ipython-input-30-8ddcc24db95a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_economia_continentes_origen['income'] = np.where(df_economia_continentes_origen['destino'].isin(income_countries),
<ipython-input-30-8ddcc24db95a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_economia_continentes_origen['development'] = np.where(df_economia_continentes_origen['destino'].isin(development_regions),


**a) Creación dataset income_by_continents**

In [31]:
# Para poder colorear los continentes en Tableau usaremos este dataset de Kaagle que contiene los codigos de países y continentes.
# Lo uniremos al que hemos creado para income. 
# Fuente: https://www.kaggle.com/datasets/joshuapaulbarnard/custom-regions-tableau-example?resource=download
df_codigos = pd.read_csv('/content/Custom-Regions-Tableau-Example.csv')

# Combinamos los dos datasets tomando como columna de union Continent y origen, ya que comparten mismo campo de valores
merged_df = df_codigos.merge(df_economia_continentes_income, left_on='Continent', right_on='origen')

df_income_by_continents = merged_df

In [32]:
# Agrupamos por origen, año e ingreso y sumar el número de migrantes
grouped_df = df_income_by_continents.groupby(['origen', 'year', 'income']).agg({'num_migrantes': 'sum'}).reset_index()

# Calculamos el total de migrantes por origen y año
total_df = df_income_by_continents.groupby(['origen', 'year']).agg({'num_migrantes': 'sum'}).reset_index()

# Unimos los dataframes grouped_df y total_df
merged_df = grouped_df.merge(total_df, on=['origen', 'year'], suffixes=('_income', '_total'))

# Calculamos el porcentaje de migrantes por tipo de ingreso
merged_df['percentage_income'] = merged_df['num_migrantes_income'] / merged_df['num_migrantes_total'] * 100

# Eliminamos las columnas innecesarias
merged_df.drop(['num_migrantes_income', 'num_migrantes_total'], axis=1, inplace=True)

# Fusionamos el dataframe original con el dataframe merged_df utilizando la columna 'origen', 'year' e 'income' como clave
df_income_by_continents = df_income_by_continents.merge(merged_df, on=['origen', 'year', 'income'], how='left')

# Reemplazamos los valores en la columna 'income'
df_income_by_continents['income'] = df_income_by_continents['income'].replace({'High-income countries': 'high', 'Middle-income countries': 'middle', 'Low-income countries': 'low'})

# Renombramos la columna 'percentage_income_x' a 'percentage_income'
df_income_by_continents.rename(columns={'percentage_income_x': 'percentage_people'}, inplace=True)

In [33]:
# Redondeamos a dos decimales
df_income_by_continents['percentage_people'] = df_income_by_continents['percentage_income'].round(2)

# Añadimos una columna con el recuento total de migrantes dado un continente y un año
origen_list = ['Europe', 'Asia', 'Africa', 'North America', 'South America', 'Oceania']
year_list = ['1990', '1995', '2000', '2005', '2010', '2015', '2020']

# Creamos una columna -total_migrantes- que calcula el número de migrantes por año y país
df_income_by_continents['total_migrantes'] = df_income_by_continents.apply(lambda row: df_income_by_continents.loc[(df_income_by_continents['origen']==row['origen']) 
                                          & (df_income_by_continents['year']==row['year']), 'num_migrantes'].unique().sum(), axis=1)

# Eliminar las columnas innecesarias
df_income_by_continents.drop(['percentage_income'], axis=1, inplace=True)

# Mostramos los 10 primeros resultados
df_income_by_continents.head(10)

,Country,Continent,Region,origen,year,num_migrantes,income,percentage_people,total_migrantes
0,Kosovo,Europe,Balkans,Europe,1990,26203905,high,55.04,47610676
1,Kosovo,Europe,Balkans,Europe,1990,20991096,middle,44.09,47610676
2,Kosovo,Europe,Balkans,Europe,1990,415675,low,0.87,47610676
3,Kosovo,Europe,Balkans,Europe,1995,28144425,high,58.05,48483589
4,Kosovo,Europe,Balkans,Europe,1995,19992803,middle,41.24,48483589
5,Kosovo,Europe,Balkans,Europe,1995,346361,low,0.71,48483589
6,Kosovo,Europe,Balkans,Europe,2000,30444071,high,61.45,49539353
7,Kosovo,Europe,Balkans,Europe,2000,18792072,middle,37.93,49539353
8,Kosovo,Europe,Balkans,Europe,2000,303210,low,0.61,49539353
9,Kosovo,Europe,Balkans,Europe,2005,32997057,high,64.53,51131462


In [34]:
# Mostramos las dimensiones
df_income_by_continents.shape

(4284, 9)

**b) Creación dataset development_by_continents**

In [35]:
# Reproducimos los pasos del ejercicio anterior; pero esta vez para los valores de development
df_codigos = pd.read_csv('/content/Custom-Regions-Tableau-Example.csv')
df_development_by_continents = df_codigos.merge(df_economia_continentes_development, left_on='Continent', right_on='origen')

In [36]:
# Agrupar por origen, año e ingreso y sumar el número de migrantes
grouped_df = df_development_by_continents.groupby(['origen', 'year', 'development']).agg({'num_migrantes': 'sum'}).reset_index()

# Calcular el total de migrantes por origen y año
total_df = df_development_by_continents.groupby(['origen', 'year']).agg({'num_migrantes': 'sum'}).reset_index()

# Unir los dataframes grouped_df y total_df
merged_df = grouped_df.merge(total_df, on=['origen', 'year'], suffixes=('_development', '_total'))

# Calcular el porcentaje de migrantes por tipo de ingreso
merged_df['percentage_development'] = merged_df['num_migrantes_development'] / merged_df['num_migrantes_total'] * 100

# Eliminar las columnas innecesarias
merged_df.drop(['num_migrantes_development', 'num_migrantes_total'], axis=1, inplace=True)

# Fusionar el dataframe original con el dataframe merged_df utilizando la columna 'origen', 'year' e 'development' como clave
df_development_by_continents = df_development_by_continents.merge(merged_df, on=['origen', 'year', 'development'], how='left')

# Reemplazar los valores en la columna 'development'
df_development_by_continents['development'] = df_development_by_continents['development'].replace({'Developed regions': 'Developed' , 'Less developed regions': 'Less developed' , 'Least developed countries': 'Least developed'})

# Renombrar la columna 'percentage_development_x' a 'percentage_development'
df_development_by_continents.rename(columns={'percentage_development_x': 'percentage_people'}, inplace=True)

In [37]:
# Redondeamos a dos decimales
df_development_by_continents['percentage_people'] = df_development_by_continents['percentage_development'].round(2)

# Añadimos una columna con el recuento total de migrantes dado un continente y un año
origen_list = ['Europe', 'Asia', 'Africa', 'North America', 'South America', 'Oceania']
year_list = ['1990', '1995', '2000', '2005', '2010', '2015', '2020']

# Creamos una columna -total_migrantes- que calcula el número de migrantes por año y país
df_development_by_continents['total_migrantes'] = df_development_by_continents.apply(lambda row: df_development_by_continents.loc[(df_development_by_continents['origen']==row['origen']) 
                                          & (df_development_by_continents['year']==row['year']), 'num_migrantes'].unique().sum(), axis=1)

# Eliminar las columnas innecesarias
df_development_by_continents.drop(['percentage_development'], axis=1, inplace=True)

# Mostramos los 10 primeros resultados
df_development_by_continents.head(10)

,Country,Continent,Region,origen,year,num_migrantes,development,percentage_people,total_migrantes
0,Kosovo,Europe,Balkans,Europe,1990,37640490,Developed,78.73,47811348
1,Kosovo,Europe,Balkans,Europe,1990,10089788,Less developed,21.10,47811348
2,Kosovo,Europe,Balkans,Europe,1990,81070,Least developed,0.17,47811348
3,Kosovo,Europe,Balkans,Europe,1995,39485417,Developed,81.03,48730769
4,Kosovo,Europe,Balkans,Europe,1995,9150586,Less developed,18.78,48730769
5,Kosovo,Europe,Balkans,Europe,1995,94766,Least developed,0.19,48730769
6,Kosovo,Europe,Balkans,Europe,2000,41342533,Developed,82.96,49833207
7,Kosovo,Europe,Balkans,Europe,2000,8385251,Less developed,16.83,49833207
8,Kosovo,Europe,Balkans,Europe,2000,105423,Least developed,0.21,49833207
9,Kosovo,Europe,Balkans,Europe,2005,43054652,Developed,83.68,51451712


In [38]:
# Mostramos las dimensiones
df_development_by_continents.shape

(4284, 9)

**Creación dataset data_gender**

In [39]:
# Realizamos las operaciones sobre el dataframe df y repetimos algunas operaciones
change_names = {
    'Region, development group, country or area of origin': 'origen',
    'Region, development group, country or area of destination': 'destino'
    }
# Aplicamos los cambios de nombre utilizando el método rename()
df_1 = df.rename(columns=change_names)

# Obtenemos las columnas que tienen el formato deseado
columnas_deseadas = [columna for columna in df.columns if columna.endswith('.1') or columna.endswith('.2')]
columnas_deseadas += ['origen', 'destino']

df_1 = df_1[columnas_deseadas]

# Mostramos las columnas filtradas
df_1.head(10)

,1990.1,1995.1,2000.1,2005.1,2010.1,2015.1,2020.1,1990.2,1995.2,2000.2,2005.2,2010.2,2015.2,2020.2,origen,destino
0,77 563 467,81 581 501,87 719 833,97 663 081,113 940 881,128 238 058,145 655 844,75 422 690,79 708 475,85 510 752,93 783 747,107 042 306,119 720 586,134 942 261,WORLD,WORLD
1,8 136 844,8 677 806,8 246 575,9 227 058,10 531 120,12 578 348,14 732 802,7 197 963,7 811 167,7 391 439,8 047 927,9 411 604,11 618 863,13 551 736,Sub-Saharan Africa,WORLD
2,8 830 013,9 829 261,10 736 407,12 097 241,14 249 463,18 177 303,20 941 050,7 535 205,8 240 176,8 700 647,9 770 550,11 064 248,14 428 744,16 622 770,Northern Africa and Western Asia,WORLD
3,17 054 672,16 042 463,17 179 372,18 895 884,24 444 167,27 971 106,31 211 577,14 049 348,13 027 520,13 718 322,14 451 806,16 638 221,18 510 024,20 017 972,Central and Southern Asia,WORLD
4,7 414 751,8 861 759,10 500 432,12 487 051,15 130 964,17 150 433,18 801 912,7 359 668,8 999 164,10 940 422,13 069 932,15 846 242,18 055 157,19 598 828,Eastern and South-Eastern Asia,WORLD
5,7 544 689,9 726 163,12 182 539,14 495 367,16 857 618,17 582 139,20 697 275,7 728 710,9 943 541,12 446 161,14 842 839,17 780 032,18 623 861,22 193 206,Latin America and the Caribbean,WORLD
6,133 623,152 358,183 319,198 892,214 866,240 815,273 803,138 894,160 703,191 857,208 246,227 037,254 405,291 478,Oceania (excluding Australia and New Zealand),WORLD
7,335 351,366 709,421 127,481 926,573 939,622 696,693 842,353 679,391 525,450 533,502 354,575 326,633 506,711 082,Australia and New Zealand,WORLD
8,23 704 534,24 148 758,24 676 104,25 682 798,27 337 252,28 229 706,31 770 385,26 827 879,27 433 406,28 201 599,29 166 292,31 343 852,32 362 696,35 831 236,Europe and Northern America,WORLD
9,24 338 984,24 834 922,25 438 034,26 505 508,28 263 791,29 203 683,32 827 157,27 487 262,28 158 769,29 020 262,30 046 328,32 323 370,33 409 329,36 988 213,Developed regions,WORLD


In [40]:
# Procedemos como en el primer caso y creamos la columnas year y num_migrantes
df_melt_1 = df_1.melt(id_vars=['origen', 'destino'], value_vars=['1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1', '2020.1',
                                                                  '1990.2', '1995.2', '2000.2', '2005.2', '2010.2', '2015.2', '2020.2'], var_name='year', value_name='num_migrantes')
# Mostramos los cinco primeros registros del DataFrame resultante
df_melt_1.head(5)

,origen,destino,year,num_migrantes
0,WORLD,WORLD,1990.1,77 563 467
1,Sub-Saharan Africa,WORLD,1990.1,8 136 844
2,Northern Africa and Western Asia,WORLD,1990.1,8 830 013
3,Central and Southern Asia,WORLD,1990.1,17 054 672
4,Eastern and South-Eastern Asia,WORLD,1990.1,7 414 751


In [41]:
# Repetimos las operaciones que hicimos en los anteriores ejercicios

# Definimos una función para obtener el género al analizar la terminación de los años
def obtener_genero(año):
    if año.endswith('.1'):
        return 'male'
    elif año.endswith('.2'):
        return 'female'
    else:
         return None

# Creamos una copia para poder iterar en el notebook sin errores
df_melt_2 = df_melt_1.copy()

# Creamos la columna "genero" en base al valor de la columna "año", si el año termina en .1 male y si termina en .2 female
df_melt_2['genero'] = df_melt_2['year'].apply(lambda x: obtener_genero(x))

# Eliminamos los caracteres ".1" o ".2" de los valores de la columna "año"
df_melt_2['year'] = df_melt_2['year'].str.replace(r'\.1|\.2', '', regex=True)

# Eliminamos los espacios en blanco al principio y final de las columnas origen y destino
df_melt_2 = df_melt_2.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Eliminamos los asteriscos que algunos nombres tienen
df_melt_2 = df_melt_2.replace(r'\*', '', regex=True)

# Eliminamos los espacios en blanco entre los números
df_melt_2['num_migrantes'] = df_melt_2['num_migrantes'].apply(lambda x: x.replace(' ', ''))

# Reemplazamos cualquier signo que no sea un número por un cero, que consideraremos como NaN
df_melt_2['num_migrantes'] = df_melt_2['num_migrantes'].replace('[^\d]+', '0', regex=True)

# Los convertimos a tipo int
df_melt_2['num_migrantes'] = df_melt_2['num_migrantes'].astype(int)

In [42]:
# Mostramos las dimensiones
df_melt_2

,origen,destino,year,num_migrantes,genero
0,WORLD,WORLD,1990,77563467,male
1,Sub-Saharan Africa,WORLD,1990,8136844,male
2,Northern Africa and Western Asia,WORLD,1990,8830013,male
3,Central and Southern Asia,WORLD,1990,17054672,male
4,Eastern and South-Eastern Asia,WORLD,1990,7414751,male
...,...,...,...,...,...
518709,French Polynesia,Wallis and Futuna Islands,2020,5,female
518710,Mayotte,Wallis and Futuna Islands,2020,9,female
518711,New Caledonia,Wallis and Futuna Islands,2020,512,female
518712,Vanuatu,Wallis and Futuna Islands,2020,28,female


**Creación de un dataframe Gender por zonas geográficas**

In [43]:
df_genero_continentes_zonas_geograficas = df_melt_2[df_melt_2['origen'].isin(zonas_geograficas) & (df_melt_2['destino'].isin(zonas_geograficas))]

In [44]:
# Diccionario de mapeo de regiones a continentes
mapeo_continentes = {
    'Eastern Africa': 'África',
    'Middle Africa': 'África',
    'Northern Africa': 'África',
    'Southern Africa': 'África',
    'Western Africa': 'África',
    'Central Asia': 'Asia',
    'Eastern Asia': 'Asia',
    'South-Eastern Asia': 'Asia',
    'Southern Asia': 'Asia',
    'Western Asia': 'Asia',
    'Eastern Europe': 'Europa',
    'Northern Europe': 'Europa',
    'Southern Europe': 'Europa',
    'Western Europe': 'Europa',
    'Caribbean': 'Latinoamerica',
    'Central America': 'Latinoamerica',
    'South America': 'Latinoamerica',
    'NORTHERN AMERICA': 'América',
    'OCEANIA': 'Oceanía'
}

# Aplicamos el mapeo de continentes al dataframe para crear una columna llamada continentes cuyos valores dependen de la zona geográfica de la columna origen
df_genero_continentes_zonas_geograficas['continentes'] = df_genero_continentes_zonas_geograficas['origen'].map(mapeo_continentes)

<ipython-input-44-f8fd36cf14e5>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genero_continentes_zonas_geograficas['continentes'] = df_genero_continentes_zonas_geograficas['origen'].map(mapeo_continentes)


In [45]:
# Creamos una copia
df_melt_4 = df_genero_continentes_zonas_geograficas.copy()

In [46]:
# Ordenamos el dataframe por origen, destino y año
df_melt_4 = df_melt_4.sort_values(['origen', 'destino', 'year'])

# Agrupamos por origen, destino y año
grouped = df_melt_4.groupby(['origen', 'destino', 'year'])

# Calculamos la diferencia entre los valores de num_migrantes
df_melt_4['diferencia'] = grouped['num_migrantes'].transform(lambda x: x[x.index[0]] - x[x.index[1]])

In [47]:
# Creamos una función que asigna hombre o mujer en función del signo de la resta que hemos hecho y guardado en el campo -predominio-
df_melt_4['predominio_migracion'] = ''
for index, row in df_melt_4.iterrows():
    if row['diferencia'] > 0:
        df_melt_4.at[index, 'predominio_migracion'] = 'hombre'
    else:
        df_melt_4.at[index, 'predominio_migracion'] = 'mujer'

In [48]:
# Convertimos los valores de diferencia a valores absolutos, para después trabajar con este valor como un factor que mida la intensidad de la diferencia. 
df_melt_4['diferencia'] = df_melt_4['diferencia'].abs()

In [49]:
df_genero_continentes_zonas_geograficas = df_melt_4

# Mostramos los cinco primeros resultados 
df_genero_continentes_zonas_geograficas.head()

,origen,destino,year,num_migrantes,genero,continentes,diferencia,predominio_migracion
27586,Caribbean,Caribbean,1990,241569,male,Latinoamerica,45961,hombre
286943,Caribbean,Caribbean,1990,195608,female,Latinoamerica,45961,hombre
64637,Caribbean,Caribbean,1995,268017,male,Latinoamerica,46632,hombre
323994,Caribbean,Caribbean,1995,221385,female,Latinoamerica,46632,hombre
101688,Caribbean,Caribbean,2000,293818,male,Latinoamerica,47071,hombre


**Creación de un dataframe Gender por países**

In [50]:
# Creamos una copia de df_melt_2
df_melt_3 = df_melt_2.copy()

In [51]:
# Ordenamos el dataframe por origen, destino y año
df_melt_3 = df_melt_3.sort_values(['origen', 'destino', 'year'])

# Agrupamos por origen, destino y año
grouped = df_melt_3.groupby(['origen', 'destino', 'year'])

# Calculamos la diferencia entre los valores de num_migrantes
df_melt_3['diferencia'] = grouped['num_migrantes'].transform(lambda x: x[x.index[0]] - x[x.index[1]])


In [52]:
# Definimos la lista con World para filtrar en destino, ya que nos interesa los datos de migrantes con destino World, de cada país. 
world = ['WORLD']
# Filtramos según la lista de países elegidos para la variable origen y para la variable destino mediante el valor world.
df_genero_countries = df_melt_3[(df_melt_3['origen'].isin(choosen_countries) & df_melt_3['destino'].isin(world))]
df_genero_countries.head(10)

,origen,destino,year,num_migrantes,genero,diferencia
130,Afghanistan,WORLD,1990,4133232,male,586882
259487,Afghanistan,WORLD,1990,3546350,female,586882
37181,Afghanistan,WORLD,1995,2372554,male,398059
296538,Afghanistan,WORLD,1995,1974495,female,398059
74232,Afghanistan,WORLD,2000,2610780,male,470883
333589,Afghanistan,WORLD,2000,2139897,female,470883
111283,Afghanistan,WORLD,2005,2282977,male,449215
370640,Afghanistan,WORLD,2005,1833762,female,449215
148334,Afghanistan,WORLD,2010,2867554,male,465590
407691,Afghanistan,WORLD,2010,2401964,female,465590


In [53]:
# Creamos una función que asigna hombre o mujer en función del signo de la resta que hemos hecho y guardado en el campo -predominio-
df_genero_countries['predominio_migracion'] = ''
for index, row in df_genero_countries.iterrows():
    if row['diferencia'] > 0:
        df_genero_countries.at[index, 'predominio_migracion'] = 'hombre'
    else:
        df_genero_countries.at[index, 'predominio_migracion'] = 'mujer'

<ipython-input-53-6979ac418742>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genero_countries['predominio_migracion'] = ''


In [54]:
# Usamos los datos del dataset -Custom-Regions-Tableau-Example- para completar con los códigos que nos permitiran representarlo en Tableau. 
# df_genero_countries_final = df_codigos.merge(df_genero_countries, left_on='Country', right_on='origen')

In [55]:
# Convertimos los valores de diferencia a valores absolutos, para después trabajar con este valor como un factor que mida la intensidad de la diferencia. 
df_genero_countries['diferencia'] = df_genero_countries['diferencia'].abs()

<ipython-input-55-6a46d2011497>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genero_countries['diferencia'] = df_genero_countries['diferencia'].abs()


In [56]:
# Eliminamos la columna que destino
df_genero_countries_final= df_genero_countries.drop('destino', axis=1)

In [57]:
# Mostramos los cinco primeros resultados
df_genero_countries_final.head()

,origen,year,num_migrantes,genero,diferencia,predominio_migracion
130,Afghanistan,1990,4133232,male,586882,hombre
259487,Afghanistan,1990,3546350,female,586882,hombre
37181,Afghanistan,1995,2372554,male,398059,hombre
296538,Afghanistan,1995,1974495,female,398059,hombre
74232,Afghanistan,2000,2610780,male,470883,hombre


In [58]:
# Mostramos las dimensiones
df_genero_countries_final.shape

(2674, 6)

**Creación de un dataframe Gender por zonas geográficas**

In [59]:
# Filtramos solo por valores de origen y destino que estén en la lista de zonas geográficas.
df_genero = df_melt_2[(df_melt['origen'].isin(zonas_geograficas) & df_melt_2['destino'].isin(zonas_geograficas))]
df_genero.head(10)

,origen,destino,year,num_migrantes,genero
6248,Eastern Africa,Eastern Africa,1990,2524242,male
6249,Middle Africa,Eastern Africa,1990,184794,male
6250,Northern Africa,Eastern Africa,1990,90526,male
6251,Southern Africa,Eastern Africa,1990,19714,male
6252,Western Africa,Eastern Africa,1990,3439,male
6254,Central Asia,Eastern Africa,1990,0,male
6255,Eastern Asia,Eastern Africa,1990,1573,male
6256,South-Eastern Asia,Eastern Africa,1990,44,male
6257,Southern Asia,Eastern Africa,1990,10030,male
6258,Western Asia,Eastern Africa,1990,175,male


In [60]:
# Añadimos coordenadas como en los casos anteriores
# Agregamos columnas de longitud y latitud al DataFrame para cada uno de los países de origen y destino
df_genero_copy = df_genero.copy()

df_genero_copy.loc[:, 'lon_origen'] = df_genero_copy['origen'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('longitud'))
df_genero_copy.loc[:, 'lat_origen'] = df_genero_copy['origen'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('latitud'))

df_genero_copy.loc[:, 'lon_destino'] = df_genero_copy['destino'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('longitud'))
df_genero_copy.loc[:, 'lat_destino'] = df_genero_copy['destino'].map(lambda x: coordenadas_zonas_geograficas.get(x, {}).get('latitud'))

# Reordenamos las columnas
column_order = ['origen', 'lon_origen', 'lat_origen', 'destino', 'lon_destino', 'lat_destino', 'year', 'num_migrantes', 'genero']
df_genero = df_genero_copy.reindex(columns=column_order)

df_genero.head(5)

,origen,lon_origen,lat_origen,destino,lon_destino,lat_destino,year,num_migrantes,genero
6248,Eastern Africa,39.0,-6.5,Eastern Africa,39.0,-6.5,1990,2524242,male
6249,Middle Africa,17.0,1.0,Eastern Africa,39.0,-6.5,1990,184794,male
6250,Northern Africa,18.0,28.0,Eastern Africa,39.0,-6.5,1990,90526,male
6251,Southern Africa,24.0,-29.0,Eastern Africa,39.0,-6.5,1990,19714,male
6252,Western Africa,-2.0,16.0,Eastern Africa,39.0,-6.5,1990,3439,male


In [61]:
# Comprobamos valores nulos en el DataFrame
valores_nulos = df_genero.isnull().sum()
# Mostrar los valores nulos por columna
print(valores_nulos)

# Contamos valores cero en el DataFrame
valores_cero = (df_genero == 0).sum()
# Mostramos los valores cero por columna
print(valores_cero)

origen           0
lon_origen       0
lat_origen       0
destino          0
lon_destino      0
lat_destino      0
year             0
num_migrantes    0
genero           0
dtype: int64
origen             0
lon_origen         0
lat_origen         0
destino            0
lon_destino        0
lat_destino        0
year               0
num_migrantes    149
genero             0
dtype: int64


In [62]:
# Mostramos los resultados
df_genero.shape

(2527, 9)

**Creación datasets flujos migratorios por zonas geográficas**

In [63]:
# Cargamos el archivo en formato .csv
df_europa = pd.read_csv("/content/Flujo_europa_1.csv", sep=';')

# Eliminamos espacios en blanco en la columna numérica
df_europa['Cantidad'] = df_europa['Cantidad'].replace(' ', '').astype(int)

# Definimos una Ruta y un nombre del archivo csv de salida
archivo_csv_europa = '/content/Flujo_europa.csv'

In [64]:
# Cargamos el archivo en formato .csv
df_asia = pd.read_csv("/content/Flujo_asia_1.csv",sep=';')

# Eliminamos espacios en blanco en la columna numérica
df_asia['Cantidad'] = df_asia['Cantidad'].replace(' ', '').astype(int)

# Definimos una Ruta y un nombre del archivo csv de salida
archivo_csv_asia= '/content/Flujo_asia.csv'

In [65]:
# Cargamos el archivo en formato .csv
df_latinoamerica = pd.read_csv("/content/Flujo_latinoamerica_1.csv", sep=';')

# Eliminamos espacios en blanco en la columna numérica
df_latinoamerica['Cantidad'] = df_latinoamerica['Cantidad'].replace(' ', '').astype(int)

# Definimos una Ruta y un nombre del archivo csv de salida
archivo_csv_latinoamerica = '/content/Flujo_latinoamerica.csv'

In [66]:
# Cargamos el archivo en formato .csv
df_africa = pd.read_csv("/content/Flujo_africa_1.csv", sep=';')

# Eliminamos espacios en blanco en la columna numérica y renombramos la columna 'Count' a 'Cantidad' como en el resto de casos
df_africa['Cantidad'] = df_africa['Count'].replace(' ', '')

# Definimos una Ruta y un nombre del archivo csv de salida
archivo_csv_africa = '/content/Flujo_africa.csv'

In [67]:
data = {
    'Año': [1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 1995, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020],
    'Origen': ['Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Australia and New Zealand', 'Melanesia', 'Melanesia', 'Melanesia', 'Melanesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Micronesia', 'Polynesia', 'Polynesia', 'Polynesia', 'Polynesia'],
    'Destino': ['Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia'],
    'Cantidad': [346087, 20394, 340, 4300, 58153, 6503, 524, 3705, 1479, 1701, 13876, 95, 93139, 10248, 730, 17809, 364148, 20791, 328, 4876, 63686, 6949, 544, 4199, 1705, 1680, 16373, 93, 97711, 11542, 670, 19840, 419804, 16506, 319, 5452, 76516, 7180, 563, 4731, 1532, 1632, 18971, 88, 113111, 12840, 610, 21853, 483479, 16663, 313, 5699, 96137, 7273, 675, 5158, 1996, 1291, 20928, 85, 121890, 11280, 597, 20955, 580866, 15001, 161, 6025, 115782, 7413, 699, 5099, 2568, 1036, 20683, 89, 132990, 9835, 367, 19890, 640591, 15144, 177, 5722, 134054, 6342, 770, 5169, 3083, 1040, 20547, 85, 146779, 13838, 387, 19522, 696555, 15282, 188, 5902, 152722, 6570, 826, 5171, 4096, 1051, 21575, 81, 163178, 14602, 407, 19500]
}

df_oceania = pd.DataFrame(data)

**Creación dataset para refugiados**

In [68]:
# Especificamos a partir de que línea del archivo csv comienzan los headers de las variables. El resto de líneas son ignoradas
numero_linea = 0
# Especificamos el separador como ";" y aplicamos un encoding para evitar problemas asociados a la decodificación de símbolos.
df = pd.read_csv("/content/data_refugiados.csv", sep= ";", header=numero_linea, encoding='latin-1')
# 
print(f"El dataset cuenta con {df.shape[0]} filas y {df.shape[1]} columnas")
# Mostramos por pantalla los 5 primeros resultados
df.head(5)

El dataset cuenta con 283 filas y 25 columnas


,ï»¿,"Region, development group, country or area",Notes,Location code,Type of data,1990,1995,2000,2005,2010,...,2005.1,2010.1,2015.1,2020.1,1990-1995,1995-2000,2000-2005,2005-2010,2010-2015,2015-2020
0,1,WORLD,NaN,900,NaN,18 977 581,18 142 316,16 503 717,13 814 075,16 153 794,...,"7,2","7,3","9,9",12,"-0,9","-1,9","-3,6","3,1","8,3","6,5"
1,2,Sub-Saharan Africa,NaN,947,NaN,4 494 798,5 076 802,3 080 836,2 659 365,2 337 822,...,"18,6","14,7","26,4","26,5","2,4",-10,"-2,9","-2,6",17,"1,5"
2,3,Northern Africa and Western Asia,NaN,1833,NaN,3 769 356,4 649 756,4 913 157,5 384 189,6 935 618,...,"23,2","21,2","26,3","27,1","4,2","1,1","1,8","5,1","9,3",4
3,4,Central and Southern Asia,NaN,921,NaN,7 011 227,3 712 870,4 277 031,2 419 247,3 506 140,...,"12,8","17,8",17,"18,4","-12,7","2,8","-11,4","7,4","-1,3","1,7"
4,5,Eastern and South-Eastern Asia,NaN,1832,NaN,460 736,466 010,612 677,503 821,513 571,...,"3,9","3,3","3,4","3,4","0,2","5,5","-3,9","0,4","3,5","1,4"


In [69]:
# Eliminamos las siguientes columnas porque no vamos a usar su información
df = df.drop("ï»¿", axis=1)
df = df.drop("Notes", axis=1)
df = df.drop("Type of data", axis=1)
df = df.drop("Location code", axis=1)

In [70]:
# Creamos una lista con los nombres de la columnas de los años; porque vamos a realizar tareas de limpieza en los valores
lista_columnas = ['1990',
 '1995',
 '2000',
 '2005',
 '2010',
 '2015',
 '2020',
 '1990.1',
 '1995.1',
 '2000.1',
 '2005.1',
 '2010.1',
 '2015.1',
 '2020.1',
 '1990-1995',
 '1995-2000',
 '2000-2005',
 '2005-2010',
 '2010-2015',
 '2015-2020']
 
# Convertimos los valores a tipo numérico y los puntos de los tipo float y eliminanos los símbolos que no sean dígitos en estos campos (por ejemplo, '~')
for column in lista_columnas:
    df[column] = df[column].apply(lambda x: re.sub(r"\D", "", str(x)))
    df[column] = df[column].replace('', '0').astype(int)

# Cambiamos el nombre de la columna 
df = df.rename(columns={"Region, development group, country or area": "country"})

In [71]:
choosen_countries = [
    "Belgium",
    "Democratic Republic of the Congo",
    "France",
    "Kenya",
    "Rwanda",
    "Uganda",
    "United Republic of Tanzania",
    "Madagascar",
    "Réunion",
    "Ethiopia",
    "Somalia",
    "Yemen",
    "Angola",
    "Burundi",
    "China",
    "Comoros",
    "India",
    "Malawi",
    "Mozambique",
    "South Africa",
    "Zimbabwe",
    "South Sudan",
    "Sudan",
    "United Kingdom",
    "Zambia",
    "Eritrea",
    "Djibouti",
    "Namibia",
    "Congo",
    "Bangladesh",
    "Canada",
    "Sri Lanka",
    "Germany",
    "Ireland",
    "Italy",
    "Japan",
    "Malaysia",
    "Pakistan",
    "Russian Federation",
    "Seychelles",
    "Spain",
    "Switzerland",
    "United States of America",
    "Cabo Verde",
    "Lesotho",
    "Nigeria",
    "Portugal",
    "Mauritius",
    "Mayotte",
    "Australia",
    "Cuba",
    "Indonesia",
    "Maldives",
    "Nepal",
    "Philippines",
    "Thailand",
    "Central African Republic",
    "Chad",
    "Egypt",
    "Libya",
    "Mali",
    "Algeria",
    "Denmark",
    "Gambia",
    "Ghana",
    "Greece",
    "Hungary",
    "Côte d'Ivoire",
    "Dem. People's Republic of Korea",
    "Republic of Korea",
    "Mauritania",
    "Netherlands",
    "Norway",
    "Senegal",
    "Sweden",
    "Togo",
    "Botswana",
    "Eswatini",
    "Cameroon",
    "Guinea",
    "Sao Tome and Principe",
    "Benin",
    "Equatorial Guinea",
    "Gabon",
    "Liberia",
    "Niger",
    "Austria",
    "Cyprus",
    "Slovenia",
    "Turkey",
    "Lebanon",
    "Israel",
    "Morocco",
    "Syrian Arab Republic",
    "Tunisia",
    "Burkina Faso",
    "State of Palestine",
    "Iraq",
    "Jordan",
    "Kuwait",
    "Saudi Arabia",
    "Western Sahara",
    "Afghanistan",
    "Albania",
    "Antigua and Barbuda",
    "Azerbaijan",
    "Argentina",
    "Bahamas",
    "Bahrain",
    "Armenia",
    "Bosnia and Herzegovina",
    "Brazil",
    "Brunei Darussalam",
    "Bulgaria",
    "Myanmar",
    "Belarus",
    "Chile",
    "Colombia",
    "Croatia",
    "Czechia",
    "Ecuador",
    "El Salvador",
    "Estonia",
    "Finland",
    "Gibraltar",
    "Guatemala",
    "Iran (Islamic Republic of)",
    "Kazakhstan",
    "Kyrgyzstan",
    "Mexico",
    "Monaco",
    "Mongolia",
    "Republic of Moldova",
    "Oman",
    "New Zealand",
    "Poland",
    "Guinea-Bissau",
    "Qatar",
    "Romania",
    "Sierra Leone",
    "Singapore",
    "Slovakia",
    "Tajikistan",
    "United Arab Emirates",
    "Turkmenistan",
    "Ukraine",
    "North Macedonia",
    "Uzbekistan",
    "Venezuela (Bolivarian Republic of)",
    "Luxembourg",
    "Barbados",
    "Bermuda",
    "Bolivia (Plurinational State of)",
    "British Virgin Islands",
    "Fiji",
    "Guyana",
    "Iceland",
    "Jamaica",
    "Latvia",
    "Lithuania",
    "Malta",
    "Nauru",
    "Nicaragua",
    "Niue",
    "Northern Mariana Islands",
    "Panama",
    "Papua New Guinea",
    "Paraguay",
    "Peru",
    "Saint Helena",
    "Saint Kitts and Nevis",
    "Serbia",
    "Trinidad and Tobago",
    "Uruguay",
    "Viet Nam",
    "Timor-Leste",
    "United States Virgin Islands",
    "Andorra",
    "Bhutan",
    "Belize",
    "Solomon Islands",
    "Cambodia",
    "Costa Rica",
    "Dominica",
    "Dominican Republic",
    "Kiribati",
    "Haiti",
    "Honduras",
    "Liechtenstein",
    "Marshall Islands",
    "Palau",
    "Saint Vincent and the"
]

# Quitamos espacios al principio y final 
df['country'] = df['country'].str.strip()

# Quitamos símbolo *
df["country"] = df["country"].str.replace("*", "")

# Filtramos los registros cuyos valores de origen y destino estén en la lista de países seleccionados
df_refugiados = df[df['country'].isin(choosen_countries)]

df_refugiados.head()

<ipython-input-71-61273b4a3f2d>:200: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["country"] = df["country"].str.replace("*", "")


,country,1990,1995,2000,2005,2010,2015,2020,1990.1,1995.1,...,2005.1,2010.1,2015.1,2020.1,1990-1995,1995-2000,2000-2005,2005-2010,2010-2015,2015-2020
23,Burundi,267929,173017,27136,40581,41427,56294,87476,804,679,...,211,168,189,254,87,371,8,4,61,88
24,Comoros,0,5,10,0,0,0,0,0,0,...,0,0,0,0,0,139,0,0,0,0
25,Djibouti,54508,27310,23257,10475,15836,22006,30794,446,274,...,114,155,196,257,138,32,16,83,66,67
26,Eritrea,0,1083,1984,6009,4946,2550,199,0,87,...,378,313,16,14,0,121,222,39,132,51
27,Ethiopia,741965,393479,197959,101026,155323,738217,734812,642,488,...,196,273,635,677,127,137,135,86,312,1


In [72]:
# Pivotamos las columnas de year y sus valores, y los convertimos en valores de dos columnas refugiados y year
df_refugiados_melt_1 = df_refugiados.melt(id_vars=['country'], value_vars=['1990', '1995', '2000', '2005', '2010', '2015', '2020'], var_name='year', value_name='num_refugiados')

df_refugiados_melt_1.head()


,country,year,num_refugiados
0,Burundi,1990,267929
1,Comoros,1990,0
2,Djibouti,1990,54508
3,Eritrea,1990,0
4,Ethiopia,1990,741965


In [73]:
df_refugiados_melt_1['year'].unique()

array(['1990', '1995', '2000', '2005', '2010', '2015', '2020'],
      dtype=object)

In [74]:
# Ahora procedemos igual, pero para las columnas con los porcentajes; es decir, las terminadas en 1990.1, 1995.1 .......

# Procedemos como en el primer caso y creamos la columnas year y num_migrantes
df_refugiados_melt_2 = df.melt(id_vars=['country'], value_vars=['1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1', '2020.1'], var_name='year', value_name='porcentajes_refugiados')

# Mostramos los cinco primeros registros del DataFrame resultante
df_refugiados_melt_2.head(5)

,country,year,porcentajes_refugiados
0,WORLD,1990.1,124
1,Sub-Saharan Africa,1990.1,338
2,Northern Africa and Western Asia,1990.1,214
3,Central and Southern Asia,1990.1,268
4,Eastern and South-Eastern Asia,1990.1,67


In [75]:
df_refugiados_melt_2['year'].unique()

array(['1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1',
       '2020.1'], dtype=object)

In [76]:
# Mover el último dígito de los valores de porcentajes_refugiados hacia la parte decimal. Podemos hacerlo porque todos los valores son tienen una parte decimal del estilo (las décimas).
df_refugiados_melt_2['porcentajes_refugiados'] = df_refugiados_melt_2['porcentajes_refugiados'] / 10

df_refugiados_melt_2

,country,year,porcentajes_refugiados
0,WORLD,1990.1,12.4
1,Sub-Saharan Africa,1990.1,33.8
2,Northern Africa and Western Asia,1990.1,21.4
3,Central and Southern Asia,1990.1,26.8
4,Eastern and South-Eastern Asia,1990.1,6.7
...,...,...,...
1976,Samoa,2020.1,0.0
1977,Tokelau,2020.1,0.0
1978,Tonga,2020.1,0.0
1979,Tuvalu,2020.1,0.0


In [77]:
df_refugiados_melt_2["year"].unique()

array(['1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1',
       '2020.1'], dtype=object)

In [78]:
# Quitamos ".1" de los valores de la columna "year"
df_refugiados_melt_2['year'] = df_refugiados_melt_2['year'].astype(str).str.rstrip('.1')

# Mostramos el resultado
df_refugiados_melt_2.head()

,country,year,porcentajes_refugiados
0,WORLD,1990,12.4
1,Sub-Saharan Africa,1990,33.8
2,Northern Africa and Western Asia,1990,21.4
3,Central and Southern Asia,1990,26.8
4,Eastern and South-Eastern Asia,1990,6.7


In [79]:
df_refugiados_melt_2['year'].unique()

array(['1990', '1995', '2000', '2005', '2010', '2015', '2020'],
      dtype=object)

In [80]:
# Convertimos la columna "year" al tipo de dato "object" en ambos DataFrames
df_refugiados_melt_1["year"] = df_refugiados_melt_1["year"].astype(str)
df_refugiados_melt_2["year"] = df_refugiados_melt_2["year"].astype(str)

# Combinamos los DataFrames en base a country y year
df_combinado = pd.merge(df_refugiados_melt_1, df_refugiados_melt_2, on=['country', 'year'], how='inner')

# Mostramos el DataFrame combinado
df_combinado

,country,year,num_refugiados,porcentajes_refugiados
0,Burundi,1990,267929,80.4
1,Comoros,1990,0,0.0
2,Djibouti,1990,54508,44.6
3,Eritrea,1990,0,0.0
4,Ethiopia,1990,741965,64.2
...,...,...,...,...
1318,Marshall Islands,2020,0,0.0
1319,Nauru,2020,1179,53.6
1320,Northern Mariana Islands,2020,0,0.0
1321,Palau,2020,0,0.0


**FASE 4. EXPORTAR ARCHIVOS A FORMATOS .CSV Y .XLSX**

In [81]:
# Exportamos los DataFrames a archivos en formato CSV

nombre_archivo = 'data_paises.csv'
df_melt.to_csv(nombre_archivo, index=False)

nombre_archivo = 'data_continentes.csv'
df_continentes.to_csv(nombre_archivo, index=False)


nombre_archivo = 'data_zonas_geograficas.csv'
df_zonas_geograficas.to_csv(nombre_archivo, index=False)

nombre_archivo = 'data_paises.csv'
df_countries.to_csv(nombre_archivo, index=False)


nombre_archivo = 'data_economia.csv'
df_economia.to_csv(nombre_archivo, index=False)

nombre_archivo = 'data_genero.csv'
df_genero.to_csv(nombre_archivo, index=False)

nombre_archivo = 'data_economia_continentes_development.csv'
df_economia_continentes_development.to_csv(nombre_archivo, index=False)

nombre_archivo = 'data_economia_continentes_income.csv'
df_economia_continentes_income.to_csv(nombre_archivo, index=False)

In [82]:
# Exportamos los dataframes a archivos Excel

nombre_archivo = 'df_continentes_final.xlsx'
df_continentes_final.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_paises.xlsx'
df_melt.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_continentes.xlsx'
df_continentes.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_zonas_geograficas.xlsx'
df_zonas_geograficas.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_paises.xlsx'
df_countries.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_economia.xlsx'
df_economia.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_genero.xlsx'
df_genero.to_excel(nombre_archivo, index=False)

nombre_archivo = 'df_income_by_continents.xlsx'
df_income_by_continents.to_excel(nombre_archivo, index=False)

nombre_archivo = 'df_development_by_continents.xlsx'
df_development_by_continents.to_excel(nombre_archivo, index=False)

nombr_archivo = 'df_genero_countries_final.xlsx'
df_genero_countries_final.to_excel(nombre_archivo, index=False)

data_economia_development_final.to_csv('data_economia_development_continentes_regiones_final.csv', index=False)
data_economia_income_final.to_csv('data_economia_income_continentes_regiones_final.csv', index=False)

nombre_archivo = 'data_genero_countries_final.xlsx'
df_genero_countries_final.to_excel(nombre_archivo, index=False)

nombre_archivo = 'df_genero_continentes_zonas_geograficas.xlsx'
df_genero_continentes_zonas_geograficas.to_excel(nombre_archivo, index=False)

nombre_archivo = 'data_refugiados_final.xlsx'
df_combinado.to_excel(nombre_archivo, index=False)

In [83]:
# Exportamos los dataframes sobre flujos migratorios dentro de continentes a archivos csv

df_europa.to_csv(archivo_csv_europa, index=False)
df_asia.to_csv(archivo_csv_asia, index=False)
df_latinoamerica.to_csv(archivo_csv_latinoamerica, index=False)
df_africa.to_csv(archivo_csv_africa, index=False)
df_oceania.to_csv('Flujo_oceania.csv', index=False)